In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as sk
from scipy.optimize import curve_fit
from scipy.stats import pearsonr
from RegscorePy import *
from scalingAnalysis import *
from IPython.core.debugger import set_trace
##### This script is to run basic scaling law analyses

# User Configs


In [2]:
years = "2000-2005"

#what features to analyze
#feats = ["Total GDP","Violent Crime","Total Robberies"]#"Number of Visitors (Estimate by Accommodation GDP)"]
feats = ["GDP","Disposable Income per Household","Unemployment"]

# feats to be plotted on a linear regression rather than log-log
linearFeats = ["Gini Index"]


# what type of connectivity to use for residual plotting    
connectivityTags = {
    #"Connectivity (asymmetric)":1#,
    "Connectivity (symmetric)":1#,
    #"Global Firm Presence (without Connectivity)":1.0
    #'Services GDP':1
    #'Connectivity (Presence times Degree)':1.0
    #"Air Traffic":1
}

# what to call this combination of connectivities in output file
connectivityLbl = "Connectivity (symmetric)"

#includeTag = 'ALL/SEPARATE' # this plots all countries on individual country plots
includeTag = 'ALL/COMBINED' # this plots all countries together
#includeTag = 'ALL/COMBINED+SEPARATE' # this plots each country individually plus all of EU as one (TODO: Not yet implemented)
#includeTag = 'LIST' # this allows manual lists of countries in "countries" above


# minimum number of datapoints to plot
dataPlotMin = 7

# what countries to analyze. 
countries = [[
'United States'
#'Australia',
#'Canada',
#'Germany',
#'Spain',
#'France',
#'United Kingdom',
#'Italy',
#'United States (Census)'
]]

# set true if running on US census data
USData = False

# max size for point in plotting correlations
maxSize = 350

# process run parameters and read in Data

In [3]:
if includeTag == 'ALL/SEPARATE':
    countries = []
    #for country in allCountries:
    countries = list(map(lambda x: [x], allCountries))
elif includeTag == 'ALL/COMBINED':
    countries = [allCountries]
elif includeTag == 'LIST':
    countries = countries
else:
    print("WHAT COUNTRIES DO YOU WANT?")

# map, mapping countryname to dataframe
data = {}

# TODO: Include error bars

# input data file
fIn = "Features"+years.replace("-","_")+".xls"


if USData:
    countries = [["United States (Census)"]]
    allCountries = countries[0]

for country in allCountries:
    data[country] = pd.read_excel(fIn,sheet_name=country)

# Run Analysis

In [4]:
# do analysis seperately for every feature
for feat in feats:

    # ignore connectivity vs connectivity
    if feat in connectivityTags.keys():
        continue


    # TODO: What about countries being plotted on linear-linear plot?
    # for every set of countries to plot together, reset residuals and connectivities to be plotted together
    for countryList in countries:
        residuals = {}
        connectivities = {}
        pops = {}
        allPops = []
        allResiduals = []
        allConnectivities = []
        countriesWithData = []
        # for each country SEPARATELY, run fits on scaling relations to get residuals, collect with connectivity data
        for country in countryList:
 
            dataToPull = ["Population",feat]
            for connectivityTag in connectivityTags.keys():
                dataToPull.append(connectivityTag)
            plotData = data[country][dataToPull].dropna()

            # skip over countryLists without data
            if plotData.empty:
                msg = 'No Data for country {} and feature {}'.format(country,feat)
                #print(msg)
                continue
                
            # skip over countryLists with less than a set number of datapoints
            if len(list(plotData.index)) < dataPlotMin:
                msg = 'Not Enough Data for country {} and feature {}'.format(country,feat)
                #print(msg)
                continue
        
            countriesWithData.append(country)

            pops[country] = plotData["Population"]
            allPops = allPops + list(pops[country])

            print("fitting ",country)
            # get scaling parameters
            Beta,y0 = getScaleParams(plotData,feat,linearFeats) 
            #plt.figure()
            ## sanitycheck for fittings
            # pop = list(plotData["Population"])
            # plt.plot(list(map(lambda x: np.log(x),pop)),list(map(lambda x: np.log(x),list(plotData[feat]))), 'ro')
            # x_space = np.linspace(np.log(np.min(pop)),np.log(np.max(pop)),len(pop))
            # plt.plot(x_space, Beta*x_space+y0, '--k')
            # plt.show()
            # plt.close()
            featVals = list(plotData[feat])

            residuals[country] = getResiduals(plotData["Population"],featVals,Beta,y0,featMean=np.average(featVals))

            

            newConnectivities = getNetConnectivities(connectivityTags,plotData)
            #newConnectivities = plotData['Ratio of Global Econ']
                
            connectivities[country] = list(newConnectivities)#list(plotData[connectivityTag])
            
            allResiduals = allResiduals + list(residuals[country])
            allConnectivities = allConnectivities + list(connectivities[country])
    
        if len(allResiduals) == 0:
            continue

        
        outDir = "Figures/ConnectivityRelations/"+years
        countryListName = ""
        if includeTag == 'ALL/COMBINED':
            countryListName = "AllCountriess"
        else:
            for country in countryList:
                app = country + "-"
                countryListName = countryListName + (app)
        if USData:
            countryListName = "US"

        outName = outDir + "/" + countryListName[:-1]+"_"+feat+"_"+connectivityLbl+ "_"+resCalc+ "PRESENTATION.png"
        #outName = outDir + "/AllCountries_" + feat+ "_Connectivity"
        plt.figure(outName)
        # 
        #residuals = connScale(residuals)
        
        maxPop = np.max(allPops)
        sizeCoeff = maxSize/maxPop
        minSize = sizeCoeff*np.min(allPops)
        cMap = plt.cm.get_cmap('hsv',len(countriesWithData)+1)
        i = 1
        # TODO: Label colors better (Hacky right now)
        for country in countriesWithData:
            first = True
            for c,r,p, in zip(connectivities[country],residuals[country],pops[country]):
                size = p*sizeCoeff
                if first:
                    plt.scatter(c,r,c=cMap(i),s=size)
                    plt.scatter(c,r,c=cMap(i),s=minSize,label=country)
                    first = False
                else:
                    plt.scatter(c,r,c=cMap(i),s=size)
            i = i + 1  

        #loggedRes = list(map(lambda x: logVals))

        rVal,pVal = pearsonr(allConnectivities,allResiduals)
        pReport = "r={}\np = {}".format(rVal,pVal)
        print(pReport)
        #plt.text(0.6,0.2,pReport,fontsize=12,transform=plt.gca().transAxes)
        xLbl = "deviation in " +connectivityLbl+" ("+resCalc+")"
        #xLbl = connectivityLbl
        yLbl = "deviation in "+feat+" ("+resCalc+")"
        plt.xlabel(xLbl)
        plt.ylabel(yLbl)
        plt.legend()
        ttl = feat + " vs. "+connectivityLbl
        plt.title(ttl)
        #plt.gca().set_yscale("log")
        #plt.gca().set_xscale("log")
        #plt.show()
        plt.savefig(outName)
        plt.close()


fitting  Canada
fitting  Germany
fitting  France
fitting  United Kingdom
fitting  Italy


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

fitting  United States


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

...

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

fitting  France
fitting  Italy
fitting  United States


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

r=-0.47961233762401007
p = 0.0011367665140414364


In [8]:
import re
# determine if two cities, with names potentially spelled differently and/or 
# in different languages, are the same cities. Cities with more than one way 
# of spelling will have them separated by '/'
def cityNameMatch(name1,name2):
    newNames = []
    for name in [name1,name2]:
        newName = name.lower()
        newName = newName.replace("(nuts 2010)","")
        newName = newName.replace("greater ","")
        newName = newName.replace(" (greater)","")
        newNameList = re.split('/|-',newName)
        newNames.append(newNameList)
        
    name1List = newNames[0]
    name2List = newNames[1]
    for subName1 in name1List:
        for subName2 in name2List:
            #if (subName1 in subName2 or subName2 in subName1) and not subName1.strip() == subName2.strip():
            #    print("edge case matching - %s , %s", (subName1, subName2))
            #if (subName1 in subName2) or (subName2 in subName1):
            if subName1.strip() == subName2.strip():
                return True
    return False






conns = pd.read_excel("Data/da11.xls").set_index("Cities")



        
    
degrees = {}
for firm in conns.columns:
    filteredPresences = conns[conns[firm] != 0]
    #print(filteredPresences[firm])
    degrees[firm] = len(list(filteredPresences[firm]))
    #newConnectivity.append(degree)
    #print(firm,nCities,len(list(conns[firm])))

    
#allCities = []
#
#for country in allCountries:
#    allCities = allCities + list(data[country]["City"])
#    
#for city in conns.index:
#    match = False
#    for loggedCity in allCities:
#        if cityNameMatch(city,loggedCity):
#            match = True
#            continue
#    if not match:
#        conns = conns.drop(city)
    
    
for city in conns.index:
    for firm in conns.columns:
        conns[firm][city] = degrees[firm] * conns[firm][city]
    
newConns = pd.DataFrame(index=conns.index,columns=["Presence times Degree"])
for city in conns.index:
    newConn = np.sum(conns.loc[city])
    newConns["Presence times Degree"][city] = newConn
    
maxVal = np.max(newConns["Presence times Degree"])

newConns['Presence times Degree'] = newConns["Presence times Degree"]/maxVal

print(newConns)

    
#plt.figure()
#plt.hist(nCities,bins=30)
#print(np.min(nCities))
#plt.xlabel("Nodal degree (Number of cities in which a firm has presence)")
#plt.ylabel("Number of firms")
#plt.savefig("Figures/ConnectivityRelations/firmNodalDegree.png")
#plt.close()

                   Presence times Degree
Cities                                  
ABERDEEN                        0.111614
ABIJAN                          0.180964
ABU DHABI                       0.206587
ACCRA                           0.169405
ADDIS ABABA                    0.0699763
ADELAIDE                        0.233115
AHMADABAD                      0.0679571
ALEXANDRIA                     0.0640579
ALGIERS                        0.0469294
ALMATY                          0.175254
AMMAN                           0.197744
AMSTERDAM                       0.591526
ANKARA                         0.0538922
ANTWERP                         0.239695
ARHUS                          0.0786102
ASUNCION                         0.17978
ATHENS                          0.360953
ATLANTA                         0.426856
AUCKLAND                        0.417073
BAGHDAD                       0.00731096
BAKU                            0.112136
BALTIMORE                       0.180302
BANDA SERI BEGAW

In [9]:
newConns.to_excel("ConnectivitiesNew.xls")